# Oil Price Prediction

## Constants

In [ ]:
BASE_PATH = 'BASE_PATH_OF_THE_PROJECT'

In [ ]:
CURRENT_PROJECT = 'check-sparsities'

## Necessary codes

In [ ]:
from google.colab import drive

drive.mount('/MyDrive', force_remount=True)

Mounted at /MyDrive


In [ ]:
!pip install ipython-autotime
%load_ext autotime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.5 MB/s eta 0:00:00
time: 435 µs (started: 2024-06-11 09:59:01 +00:00)


## Helper Functions

In [ ]:
def flatten(a):
  isChanged = None
  b = []
  while True:
    isChanged = False
    for x in a:
      try:
        b = b + list(x)
        isChanged = True
      except:
        b.append(x)
    a = b
    b = []
    if not isChanged:
      break
  return a

time: 4.36 ms (started: 2024-06-11 09:59:01 +00:00)


In [ ]:
def show_models_results(model, train_input, train_output, test_input, test_output, squeeze=True):
  def check_and_calc_squeeze(inp):
    if squeeze:
      return np.squeeze(inp)
    else:
      return inp

  print("train and test MAE: ",
        mean_absolute_error(train_output, model.predict(check_and_calc_squeeze(train_input), verbose=0)),
        mean_absolute_error(test_output, model.predict(check_and_calc_squeeze(test_input), verbose=0)))
  print("train and test RMSE: ",
        mean_squared_error(train_output, model.predict(check_and_calc_squeeze(train_input), verbose=0), squared=False),
        mean_squared_error(test_output, model.predict(check_and_calc_squeeze(test_input), verbose=0), squared=False))
  print("train and test MSE: ",
        mean_squared_error(train_output, model.predict(check_and_calc_squeeze(train_input), verbose=0)),
        mean_squared_error(test_output, model.predict(check_and_calc_squeeze(test_input), verbose=0)))
  print("train and test MAPE: ",
        mean_absolute_percentage_error(train_output, model.predict(check_and_calc_squeeze(train_input), verbose=0)),
        mean_absolute_percentage_error(test_output, model.predict(check_and_calc_squeeze(test_input), verbose=0)))
  print("train and test R2: ",
        r2_score(train_output, model.predict(check_and_calc_squeeze(train_input), verbose=0)),
      r2_score(test_output, model.predict(check_and_calc_squeeze(test_input), verbose=0)))


time: 1.43 ms (started: 2024-06-11 09:59:01 +00:00)


In [ ]:
def show_models_results_with_labels(y_true, y_pred):
  print("MAE: ",
        mean_absolute_error(y_true, y_pred))
  print("RMSE: ",
        mean_squared_error(y_true, y_pred, squared=False))
  print("MSE: ",
        mean_squared_error(y_true, y_pred))
  print("MAPE: ",
        mean_absolute_percentage_error(y_true, y_pred))
  print("R2: ",
        r2_score(y_true, y_pred))


time: 809 µs (started: 2024-06-11 09:59:01 +00:00)


In [ ]:
def write_models_and_cluster(models, cluster, model_name, trial_number):
  main_writing_path = BASE_PATH + 'models/' + CURRENT_PROJECT + "/" + model_name + "/" + str(trial_number)
  cluster_path = main_writing_path + '/cluster'
  models_path = main_writing_path + "/models"

  os.makedirs(cluster_path)
  pickle.dump(cluster, open(cluster_path + '/cluster_file', 'wb'))

  for i, model in enumerate(models):
    current_model_path = models_path + "/" + str(i)
    os.makedirs(current_model_path)
    model.save(current_model_path)

time: 884 µs (started: 2024-06-11 09:59:01 +00:00)


In [ ]:
def write_model(model, model_name, trial_number):
  main_writing_path = BASE_PATH + 'models/' + CURRENT_PROJECT + "/" + model_name + "/" + str(trial_number)
  model_path = main_writing_path + "/model"

  os.makedirs(model_path)

  model.save(model_path)

time: 712 µs (started: 2024-06-11 09:59:01 +00:00)


In [ ]:
def read_models_and_cluster(model_name, trial_number):
  main_reading_path = BASE_PATH + 'models/' + CURRENT_PROJECT + "/" + model_name + "/" + str(trial_number)
  cluster_path = main_reading_path + '/cluster'
  models_path = main_reading_path + "/models"

  cluster = pickle.load(open(cluster_path + '/cluster_file', 'rb'))

  models = []

  for model in (os.listdir(models_path)):
    current_model_path = models_path + "/" + model
    models.append(tf.keras.models.load_model(current_model_path))

  return cluster, models

time: 1.43 ms (started: 2024-06-11 09:59:01 +00:00)


In [ ]:
def read_model(model_name, trial_number):
  main_reading_path = BASE_PATH + 'models/' + CURRENT_PROJECT + "/" + model_name + "/" + str(trial_number)
  model_path = main_reading_path + "/model"

  model = tf.keras.models.load_model(model_path)

  return model

time: 882 µs (started: 2024-06-11 09:59:01 +00:00)


In [ ]:
def is_model_trained(model_name, trial_number):
  main_reading_path = BASE_PATH + 'models/' + CURRENT_PROJECT + "/" + model_name + "/" + str(trial_number)
  return os.path.exists(main_reading_path)

time: 768 µs (started: 2024-06-11 09:59:01 +00:00)


In [ ]:
def write_model_results(model_name, trial_number, y_train_pred, y_test_pred):
  main_writing_path = BASE_PATH + 'results/' + CURRENT_PROJECT + "/" + model_name + "/" + str(trial_number)
  os.makedirs(main_writing_path)
  pd.DataFrame({'predictions': y_train_pred.squeeze()}).to_csv(main_writing_path + "/train.csv", index=False)
  pd.DataFrame({'predictions': y_test_pred.squeeze()}).to_csv(main_writing_path + "/test.csv", index=False)

time: 901 µs (started: 2024-06-11 09:59:01 +00:00)


In [ ]:
def read_model_results(model_name, trial_number):
  main_writing_path = BASE_PATH + 'results/' + CURRENT_PROJECT + "/" + model_name + "/" + str(trial_number)
  train = pd.read_csv(main_writing_path + "/train.csv")
  test = pd.read_csv(main_writing_path + "/test.csv")
  return train, test

time: 715 µs (started: 2024-06-11 09:59:01 +00:00)


## Imports

In [ ]:
!pip install tslearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 5.0 MB/s eta 0:00:00
time: 12.9 s (started: 2024-06-11 09:59:01 +00:00)


In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import copy
from sklearn.cluster import KMeans
from tslearn.clustering import TimeSeriesKMeans
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score
import random
from keras.constraints import Constraint
from keras import backend as kerasBackend
import seaborn as sns
import pickle
import os

time: 22.6 s (started: 2024-06-11 09:59:14 +00:00)


In [ ]:
sns.set()

time: 7.88 ms (started: 2024-06-11 09:59:37 +00:00)


## Data Loading and Preparation

### Constants

In [ ]:
START_DATE = '01/04/2010'
END_DATE = '07/31/2020'
WTI_DATASET_PATH = BASE_PATH + 'data/processed-WTI-price-daily-investing.csv'
BRENT_DATASET_PATH = BASE_PATH + 'data/processed-Brent-price-daily-investing.csv'
TRAIN_PORTION = 0.7
VALIDATION_PORTION = 0
TEST_PORTION = 0.3
WINDOW_SIZE = 5

time: 18.6 ms (started: 2024-06-11 09:59:37 +00:00)


### Load data

In [ ]:
wti_whole = pd.read_csv(WTI_DATASET_PATH)
wti_whole

,Date,Price
0,01/04/2010,81.51
1,01/05/2010,81.77
2,01/06/2010,83.18
3,01/07/2010,82.66
4,01/08/2010,82.75
...,...,...
2741,07/27/2020,41.60
2742,07/28/2020,41.04
2743,07/29/2020,41.27
2744,07/30/2020,39.92


time: 973 ms (started: 2024-06-11 09:59:37 +00:00)


In [ ]:
brent_whole = pd.read_csv(BRENT_DATASET_PATH)
brent_whole

,Date,Price
0,01/04/2010,80.12
1,01/05/2010,80.59
2,01/06/2010,81.89
3,01/07/2010,81.51
4,01/08/2010,81.37
...,...,...
2728,07/27/2020,43.41
2729,07/28/2020,43.22
2730,07/29/2020,43.75
2731,07/30/2020,42.94


time: 403 ms (started: 2024-06-11 09:59:38 +00:00)


### Get Wanted Period

In [ ]:
wti_intersection = copy.deepcopy(wti_whole)
# wti_intersection = wti_whole[wti_whole["Date"] >= START_DATE]
# wti_intersection = wti_intersection[wti_intersection["Date"] <= END_DATE]
# assert (wti_intersection.iloc[0]["Date"] == START_DATE)
# assert (wti_intersection.iloc[-1]["Date"] == END_DATE)

time: 2.02 ms (started: 2024-06-11 09:59:38 +00:00)


In [ ]:
brent_intersection = copy.deepcopy(brent_whole)
# brent_intersection = brent_whole[brent_whole["Date"] >= START_DATE]
# brent_intersection = brent_intersection[brent_intersection["Date"] <= END_DATE]
# assert (brent_intersection.iloc[0]["Date"] == START_DATE)
# assert (brent_intersection.iloc[-1]["Date"] == END_DATE)

time: 3.8 ms (started: 2024-06-11 09:59:38 +00:00)


### Get mean and standard deviation for normalization

In [ ]:
wti_mean = np.mean(wti_intersection['Price'])
wti_std = np.std(wti_intersection['Price'])
print("WTI mean and std: ", wti_mean, wti_std)

WTI mean and std:  70.20891478514203 22.941086577893046
time: 12 ms (started: 2024-06-11 09:59:38 +00:00)


In [ ]:
brent_mean = np.mean(brent_intersection['Price'])
brent_std = np.std(brent_intersection['Price'])
print("Brent mean and std: ", brent_mean, brent_std)

Brent mean and std:  77.96642151481889 26.59954316383742
time: 5.52 ms (started: 2024-06-11 09:59:38 +00:00)


### Divide into training and test

In [ ]:
number_of_datapoints_wti = len(wti_intersection)
number_of_datapoints_brent = len(brent_intersection)
number_of_datapoints_wti, number_of_datapoints_brent

(2746, 2733)

time: 6.29 ms (started: 2024-06-11 09:59:38 +00:00)


In [ ]:
train_part_wti = math.floor(number_of_datapoints_wti * TRAIN_PORTION)
validation_part_wti = math.floor(number_of_datapoints_wti * VALIDATION_PORTION)
test_part_wti = math.floor(number_of_datapoints_wti * TEST_PORTION)

time: 1.86 ms (started: 2024-06-11 09:59:39 +00:00)


In [ ]:
train_part_brent = math.floor(number_of_datapoints_brent * TRAIN_PORTION)
validation_part_brent = math.floor(number_of_datapoints_brent * VALIDATION_PORTION)
test_part_brent = math.floor(number_of_datapoints_brent * TEST_PORTION)

time: 911 µs (started: 2024-06-11 09:59:39 +00:00)


In [ ]:
wti_train = wti_intersection.iloc[:train_part_wti].reset_index(drop=True)
wti_validation = wti_intersection.iloc[train_part_wti:train_part_wti+validation_part_wti].reset_index(drop=True)
wti_test = wti_intersection.iloc[train_part_wti+validation_part_wti:].reset_index(drop=True)
print('number of records of parts: ', [len(wti_train), len(wti_validation), len(wti_test)])

number of records of parts:  [1922, 0, 824]
time: 3.93 ms (started: 2024-06-11 09:59:39 +00:00)


In [ ]:
brent_train = brent_intersection.iloc[:train_part_brent].reset_index(drop=True)
brent_validation = brent_intersection.iloc[train_part_brent:train_part_brent+validation_part_brent].reset_index(drop=True)
brent_test = brent_intersection.iloc[train_part_brent+validation_part_brent:].reset_index(drop=True)
print('number of records of parts: ', [len(brent_train), len(brent_validation), len(brent_test)])

number of records of parts:  [1913, 0, 820]
time: 17.5 ms (started: 2024-06-11 09:59:39 +00:00)


### Prepare inputs and targets

In [ ]:
import tensorflow as tf
import numpy as np
def get_input_and_output_tensor(df, feature_column, target_column, window_size):
  """
  inputs:
    df -> pandas dataframe
    feature_column -> column that is our feature
    target_column -> column that is our target
    window_size -> window size of inputs
  output:
    [inputs: (m ,window_size, 1), outputs: (m)]
  """
  inputs = np.zeros((len(df[feature_column]) - window_size, window_size, 1))
  outputs = np.zeros((len(df[feature_column]) - window_size))

  start = 0
  for i in range(window_size, len(df[feature_column])):
    inputs[start,:,0] = df[feature_column][start:i]
    outputs[start] = df[target_column][i]
    start = start + 1

  inputs = tf.convert_to_tensor(inputs)
  outputs = tf.convert_to_tensor(outputs)

  return [inputs, outputs]

time: 1.35 ms (started: 2024-06-11 09:59:39 +00:00)


In [ ]:
wti_train_inputs, wti_train_outputs = get_input_and_output_tensor(wti_train, "Price", "Price", WINDOW_SIZE)
if len(wti_validation) > 0:
  wti_validation_inputs, wti_validation_outputs = get_input_and_output_tensor(wti_validation, "Price", "Price", WINDOW_SIZE)
wti_test_inputs, wti_test_outputs = get_input_and_output_tensor(wti_test, "Price", "Price", WINDOW_SIZE)
print("train: ", wti_train_inputs.shape, wti_train_outputs.shape)
if len(wti_validation) > 0:
  print("validation: ", wti_validation_inputs.shape, wti_validation_outputs.shape)
print("test: ", wti_test_inputs.shape, wti_test_outputs.shape)

train:  (1917, 5, 1) (1917,)
test:  (819, 5, 1) (819,)
time: 855 ms (started: 2024-06-11 09:59:39 +00:00)


In [ ]:
brent_train_inputs, brent_train_outputs = get_input_and_output_tensor(brent_train, "Price", "Price", WINDOW_SIZE)
if len(brent_validation) > 0:
  brent_validation_inputs, brent_validation_outputs = get_input_and_output_tensor(brent_validation, "Price", "Price", WINDOW_SIZE)
brent_test_inputs, brent_test_outputs = get_input_and_output_tensor(brent_test, "Price", "Price", WINDOW_SIZE)
print("train: ", brent_train_inputs.shape, brent_train_outputs.shape)
if len(wti_validation) > 0:
  print("validation: ", brent_validation_inputs.shape, brent_validation_outputs.shape)
print("test: ", brent_test_inputs.shape, brent_test_outputs.shape)

train:  (1908, 5, 1) (1908,)
test:  (815, 5, 1) (815,)
time: 339 ms (started: 2024-06-11 09:59:40 +00:00)


### Normalize inputs

In [ ]:
wti_train_inputs = wti_train_inputs - wti_mean
wti_train_inputs = wti_train_inputs / wti_std

if len(wti_validation) > 0:
  wti_validation_inputs = wti_validation_inputs - wti_mean
  wti_validation_inputs = wti_validation_inputs / wti_std

wti_test_inputs = wti_test_inputs - wti_mean
wti_test_inputs = wti_test_inputs / wti_std

print("train: ", wti_train_inputs.shape, np.min(wti_train_inputs), np.max(wti_train_inputs))
if len(wti_validation) > 0:
  print("validation: ", wti_validation_inputs.shape, np.min(wti_validation_inputs), np.max(wti_validation_inputs))
print("test: ", wti_test_inputs.shape, np.min(wti_test_inputs), np.max(wti_test_inputs))

train:  (1917, 5, 1) -1.917908928844772 1.9057983616604008
test:  (819, 5, 1) -4.700689063658387 0.270304773655908
time: 24.1 ms (started: 2024-06-11 09:59:40 +00:00)


In [ ]:
brent_train_inputs = brent_train_inputs - brent_mean
brent_train_inputs = brent_train_inputs / brent_std

if len(wti_validation) > 0:
  brent_validation_inputs = brent_validation_inputs - brent_mean
  brent_validation_inputs = brent_validation_inputs / brent_std

brent_test_inputs = brent_test_inputs - brent_mean
brent_test_inputs = brent_test_inputs / brent_std

print("train: ", brent_train_inputs.shape, np.min(brent_train_inputs), np.max(brent_train_inputs))
if len(wti_validation) > 0:
  print("validation: ", brent_validation_inputs.shape, np.min(brent_validation_inputs), np.max(brent_validation_inputs))
print("test: ", brent_test_inputs.shape, np.min(brent_test_inputs), np.max(brent_test_inputs))

train:  (1908, 5, 1) -1.8829805161057174 1.8302411505836444
test:  (815, 5, 1) -2.204414607937185 0.3129218586166238
time: 13.2 ms (started: 2024-06-11 09:59:40 +00:00)


## Experiments

### Compile and Fit parameters

In [ ]:
compile_parameters = {
    "optimizer": "adam",
    "loss": "mae"
}

fit_parameters = {
    "batch_size": 64,
    "epochs": 20,
    "shuffle": True,
    "verbose": 0,
}

time: 1.01 ms (started: 2024-06-11 09:59:40 +00:00)


### Constants

In [ ]:
NUMBER_OF_TRIALS = 10

time: 691 µs (started: 2024-06-11 09:59:40 +00:00)


### Kmeans-LSTM-Euclidean

#### Model Name

In [ ]:
MODEL_NAME = 'DSD-LSTM'

time: 492 µs (started: 2024-06-11 09:59:40 +00:00)


#### DSD Classes and Functions

In [ ]:
class Sparse(Constraint):
    '''
    We will use one variable: Mask
    After we train our model dense model,
    we will save the weights and analyze them.
    We will create a mask where 1 means the
    number is far away enough from 0 and 0
    if it is to close to 0. We will multiply
    the weights by 0(making them 0) if they
    are supposed to be masked.
    '''

    def __init__(self, mask):
        self.mask = kerasBackend.cast_to_floatx(mask)

    def __call__(self,x):
        return self.mask * x

    def get_config(self):
        return {'mask': self.mask}

time: 855 µs (started: 2024-06-11 09:59:40 +00:00)


In [ ]:
def create_sparsity_masks(model,sparsity):
    weights_list = model.get_weights()
    masks = []
    for weights in weights_list:
        #We can ignore biases
        if len(weights.shape) > 1:
            weights_abs = np.abs(weights)
            masks.append((weights_abs>np.percentile(weights_abs,sparsity))*1.)
    return masks

time: 837 µs (started: 2024-06-11 09:59:40 +00:00)


#### Make the main models and fit

In [ ]:
# best_wti_model = None
# best_wti_errors = [None, None]
# best_wti_history = None
# best_brent_model = None
# best_brent_errors = [None, None]
# best_brent_history = None

# fig, axis = plt.subplots(NUMBER_OF_TRIALS * 2, 3)

customized_fit_parameters = copy.deepcopy(fit_parameters)
customized_fit_parameters['epochs'] = 50

for sparsity_value in [25, 35, 45, 55]:
  best_wti_model = None
  best_wti_error = None
  best_wti_predictions = None

  best_brent_model = None
  best_brent_error = None
  best_brent_predictions = None

  if is_model_trained(MODEL_NAME + "_WTI" + str(sparsity_value), 'best') or is_model_trained(MODEL_NAME + "_BRENT" + str(sparsity_value), 'best'):
    print('all models of sparsity = ', sparsity_value, 'have been trained!')
    continue

  print('\n\ncurrent sparsity value: ', sparsity_value)
  for _ in range(NUMBER_OF_TRIALS):
    print("start")

# ADDED
    if is_model_trained(MODEL_NAME + "_WTI" + str(sparsity_value), _) or is_model_trained(MODEL_NAME + "_BRENT" + str(sparsity_value), _):
      print('the model had been trained')

      brent_train_error, brent_test_error = read_model_results(MODEL_NAME + "_BRENT" + str(sparsity_value), _)
      wti_train_error, wti_test_error = read_model_results(MODEL_NAME + "_WTI" + str(sparsity_value), _)
      brent_model = read_model(MODEL_NAME + "_BRENT" + str(sparsity_value), _)
      wti_model = read_model(MODEL_NAME + "_WTI" + str(sparsity_value), _)

      print('brent train: ')
      show_models_results_with_labels(brent_train_outputs, brent_train_error)
      print('brent test: ')
      show_models_results_with_labels(brent_test_outputs, brent_test_error)
      print("-"*20)
      print('wti train: ')
      show_models_results_with_labels(wti_train_outputs, wti_train_error)
      print('wti test: ')
      show_models_results_with_labels(wti_test_outputs, wti_test_error)

      wti_test_mae = mean_absolute_error(wti_test_outputs, wti_test_error)
      brent_test_mae = mean_absolute_error(brent_test_outputs, brent_test_error)

      if best_wti_model == None or wti_test_mae < best_wti_error:
        best_wti_model = wti_model
        best_wti_error = wti_test_mae
        best_wti_predictions = [wti_train_error, wti_test_error]

      if best_brent_model == None or brent_test_mae < best_brent_error:
        best_brent_model = brent_model
        best_brent_error = brent_test_mae
        best_brent_predictions = [brent_train_error, brent_test_error]

      print("END\n\n")
      continue


    wti_sizes = [random.randint(32, 512) for i in range(3)]
    brent_sizes = [random.randint(32, 512) for i in range(3)]

    # Dense
    wti_model = tf.keras.Sequential([
        tf.keras.layers.LSTM(128, input_shape=(WINDOW_SIZE, 1), return_sequences=True),
        tf.keras.layers.LSTM(128),
        # tf.keras.layers.Dense(wti_sizes[0], 'leaky_relu'),
        # tf.keras.layers.Dense(wti_sizes[1], 'leaky_relu'),
        tf.keras.layers.Dense(wti_sizes[2], 'leaky_relu'),
        tf.keras.layers.Dense(1),
    ])
    brent_model = tf.keras.Sequential([
        tf.keras.layers.LSTM(128, input_shape=(WINDOW_SIZE, 1), return_sequences=True),
        tf.keras.layers.LSTM(128),
        # tf.keras.layers.Dense(brent_sizes[0], 'leaky_relu'),
        # tf.keras.layers.Dense(brent_sizes[1], 'leaky_relu'),
        tf.keras.layers.Dense(brent_sizes[2], 'leaky_relu'),
        tf.keras.layers.Dense(1),
    ])

    wti_model.compile(**compile_parameters)
    brent_model.compile(**compile_parameters)

    wti_model.fit((wti_train_inputs), wti_train_outputs, **customized_fit_parameters,
                              validation_data=((wti_test_inputs), wti_test_outputs))
    brent_model.fit((brent_train_inputs), brent_train_outputs, **customized_fit_parameters,
                              validation_data=((brent_test_inputs), brent_test_outputs))

    print("WTI Dense")
    show_models_results(wti_model, wti_train_inputs, wti_train_outputs,
                        wti_test_inputs, wti_test_outputs, squeeze=False)
    print("Brent Dense")
    show_models_results(brent_model, brent_train_inputs, brent_train_outputs,
                        brent_test_inputs, brent_test_outputs, squeeze=False)
    # sns.histplot(flatten(wti_model.get_weights()), bins=(80), kde=True, ax=axis[_*2,0])
    # sns.histplot(flatten(brent_model.get_weights()), bins=(80), kde=True, ax=axis[_*2+1,0])

    # Sparse
    wti_masks = create_sparsity_masks(wti_model, sparsity_value)
    brent_masks = create_sparsity_masks(brent_model, sparsity_value)

    wti_sparse_model = tf.keras.Sequential([
        tf.keras.layers.LSTM(128, input_shape=(WINDOW_SIZE, 1), return_sequences=True,
                             kernel_constraint=(Sparse(wti_masks[0])),
                             recurrent_constraint=(Sparse(wti_masks[1]))),
        tf.keras.layers.LSTM(128,
                             kernel_constraint=(Sparse(wti_masks[2])),
                             recurrent_constraint=(Sparse(wti_masks[3]))),
        # tf.keras.layers.Dense(wti_sizes[0], 'leaky_relu'),
        # tf.keras.layers.Dense(wti_sizes[1], 'leaky_relu'),
        tf.keras.layers.Dense(wti_sizes[2], 'leaky_relu',
                              kernel_constraint=Sparse(wti_masks[4])),
        tf.keras.layers.Dense(1,
                              kernel_constraint=Sparse(wti_masks[5])),
    ])
    brent_sparse_model = tf.keras.Sequential([
        tf.keras.layers.LSTM(128, input_shape=(WINDOW_SIZE, 1), return_sequences=True,
                            kernel_constraint=(Sparse(brent_masks[0])),
                            recurrent_constraint=(Sparse(brent_masks[1]))),
        tf.keras.layers.LSTM(128,
                            kernel_constraint=(Sparse(brent_masks[2])),
                            recurrent_constraint=(Sparse(brent_masks[3]))),
        # tf.keras.layers.Dense(brent_sizes[0], 'leaky_relu'),
        # tf.keras.layers.Dense(brent_sizes[1], 'leaky_relu'),
        tf.keras.layers.Dense(brent_sizes[2], 'leaky_relu',
                              kernel_constraint=Sparse(brent_masks[4])),
        tf.keras.layers.Dense(1,
                              kernel_constraint=Sparse(brent_masks[5])),
    ])

    wti_sparse_model.compile(**compile_parameters)
    brent_sparse_model.compile(**compile_parameters)

    wti_sparse_model.set_weights(wti_model.get_weights())
    brent_sparse_model.set_weights(brent_model.get_weights())

    wti_model_history = wti_sparse_model.fit((wti_train_inputs), wti_train_outputs, **customized_fit_parameters,
                              validation_data=((wti_test_inputs), wti_test_outputs))
    brent_model_history = brent_sparse_model.fit((brent_train_inputs), brent_train_outputs, **customized_fit_parameters,
                              validation_data=((brent_test_inputs), brent_test_outputs))

    # sns.histplot(flatten(wti_sparse_model.get_weights()), bins=(80), kde=True, ax=axis[_*2,1])
    # sns.histplot(flatten(brent_sparse_model.get_weights()), bins=(80), kde=True, ax=axis[_*2+1,1])

    # re-Dense
    wti_redense_model = tf.keras.Sequential([
        tf.keras.layers.LSTM(128, input_shape=(WINDOW_SIZE, 1), return_sequences=True),
        tf.keras.layers.LSTM(128),
        # tf.keras.layers.Dense(wti_sizes[0], 'leaky_relu'),
        # tf.keras.layers.Dense(wti_sizes[1], 'leaky_relu'),
        tf.keras.layers.Dense(wti_sizes[2], 'leaky_relu'),
        tf.keras.layers.Dense(1),
    ])
    brent_redense_model = tf.keras.Sequential([
        tf.keras.layers.LSTM(128, input_shape=(WINDOW_SIZE, 1), return_sequences=True),
        tf.keras.layers.LSTM(128),
        # tf.keras.layers.Dense(brent_sizes[0], 'leaky_relu'),
        # tf.keras.layers.Dense(brent_sizes[1], 'leaky_relu'),
        tf.keras.layers.Dense(brent_sizes[2], 'leaky_relu'),
        tf.keras.layers.Dense(1),
    ])

    wti_compile_parameters = copy.deepcopy(compile_parameters)
    wti_compile_parameters["optimizer"] = tf.keras.optimizers.Adam(0.0001)
    brent_compile_parameters = copy.deepcopy(compile_parameters)
    brent_compile_parameters["optimizer"] = tf.keras.optimizers.Adam(0.0001)

    wti_redense_model.compile(**wti_compile_parameters)
    brent_redense_model.compile(**brent_compile_parameters)

    wti_redense_model.set_weights(wti_sparse_model.get_weights())
    brent_redense_model.set_weights(brent_sparse_model.get_weights())

    wti_model_history = wti_redense_model.fit((wti_train_inputs), wti_train_outputs, **customized_fit_parameters,
                              validation_data=((wti_test_inputs), wti_test_outputs))
    brent_model_history = brent_redense_model.fit((brent_train_inputs), brent_train_outputs, **customized_fit_parameters,
                              validation_data=((brent_test_inputs), brent_test_outputs))

    # sns.histplot(flatten(wti_redense_model.get_weights()), bins=(80), kde=True, ax=axis[_*2,2])
    # sns.histplot(flatten(brent_redense_model.get_weights()), bins=(80), kde=True, ax=axis[_*2+1,2])

    print("-"*20)
    print("WTI re-Dense")
    show_models_results(wti_redense_model, wti_train_inputs, wti_train_outputs,
                        wti_test_inputs, wti_test_outputs, squeeze=False)
    print("Brent re-Dense")
    show_models_results(brent_redense_model, brent_train_inputs, brent_train_outputs,
                        brent_test_inputs, brent_test_outputs, squeeze=False)

    wti_model = wti_redense_model
    brent_model = brent_redense_model

    wti_train_predicted_labels = wti_model.predict(wti_train_inputs)
    wti_test_predicted_labels = wti_model.predict(wti_test_inputs)
    brent_train_predicted_labels = brent_model.predict(brent_train_inputs)
    brent_test_predicted_labels = brent_model.predict(brent_test_inputs)

    wti_test_mae = mean_absolute_error(wti_test_outputs, wti_test_predicted_labels)
    brent_test_mae = mean_absolute_error(brent_test_outputs, brent_test_predicted_labels)

# ADDED
    write_model(wti_model, MODEL_NAME + "_WTI" + str(sparsity_value), _)
    write_model_results(MODEL_NAME + "_WTI" + str(sparsity_value), _, wti_train_predicted_labels, wti_test_predicted_labels)
    write_model(brent_model, MODEL_NAME + "_BRENT" + str(sparsity_value), _)
    write_model_results(MODEL_NAME + "_BRENT" + str(sparsity_value), _, brent_train_predicted_labels, brent_test_predicted_labels)
# END_ADDED

    if best_wti_model == None or wti_test_mae < best_wti_error:
      best_wti_model = wti_model
      best_wti_error = wti_test_mae
      best_wti_predictions = [wti_train_predicted_labels, wti_test_predicted_labels]

    if best_brent_model == None or brent_test_mae < best_brent_error:
      best_brent_model = brent_model
      best_brent_error = brent_test_mae
      best_brent_predictions = [brent_train_predicted_labels, brent_test_predicted_labels]

  if not (is_model_trained(MODEL_NAME + "_WTI" + str(sparsity_value), 'best') or is_model_trained(MODEL_NAME + "_BRENT" + str(sparsity_value), 'best')):
    write_model(best_wti_model, MODEL_NAME + "_WTI" + str(sparsity_value), 'best')
    write_model_results(MODEL_NAME + "_WTI" + str(sparsity_value), 'best', best_wti_predictions[0], best_wti_predictions[1])
    write_model(best_brent_model, MODEL_NAME + "_BRENT" + str(sparsity_value), 'best')
    write_model_results(MODEL_NAME + "_BRENT" + str(sparsity_value), 'best', best_brent_predictions[0], best_brent_predictions[1])

# wti_model = best_wti_model
# brent_model = best_brent_model
# wti_model_history = best_wti_history
# brent_model_history = best_brent_history

all models of sparsity =  25 have been trained!
all models of sparsity =  35 have been trained!
all models of sparsity =  45 have been trained!
all models of sparsity =  55 have been trained!
time: 1.02 s (started: 2024-06-11 09:59:40 +00:00)


#### Show Results

In [ ]:
for sparsity_value in [25, 35, 45, 55]:
  wti_train, wti_test = read_model_results(MODEL_NAME + "_WTI" + str(sparsity_value), 'best')
  brent_train, brent_test = read_model_results(MODEL_NAME + "_BRENT" + str(sparsity_value), 'best')

  print("sparsity: ", sparsity_value)
  print("WTI")
  show_models_results_with_labels(wti_test_outputs, wti_test)
  print("Brent")
  show_models_results_with_labels(brent_test_outputs, brent_test)
  print("-"*20)
  print

sparsity:  25
WTI
MAE:  0.9747547807081806
RMSE:  2.5022248671854896
MSE:  6.261129285961442
MAPE:  0.023683925418020658
R2:  0.9584498591587102
Brent
MAE:  0.9116918871165645
RMSE:  1.3608099439699817
MSE:  1.851803703607585
MAPE:  0.01747625382126151
R2:  0.9889944526361812
--------------------
sparsity:  35
WTI
MAE:  0.980935427350427
RMSE:  2.4362693510683036
MSE:  5.935408350954773
MAPE:  0.02256308463471193
R2:  0.9606114102314263
Brent
MAE:  0.9147490638036813
RMSE:  1.352749555200233
MSE:  1.8299313590944284
MAPE:  0.017535475950660545
R2:  0.9891244432626328
--------------------
sparsity:  45
WTI
MAE:  0.995815131257631
RMSE:  2.4652506674368118
MSE:  6.077460853297646
MAPE:  0.023414965874532196
R2:  0.9596687206286995
Brent
MAE:  0.9280681411042945
RMSE:  1.3510535157942771
MSE:  1.8253456025400772
MAPE:  0.017576916611049428
R2:  0.9891516971021513
--------------------
sparsity:  55
WTI
MAE:  0.9756848998778996
RMSE:  2.4490340698015647
MSE:  5.997767875048815
MAPE:  0.0227

In [ ]:
sparsity_value = 25

for _ in range(10):
  wti_train, wti_test = read_model_results(MODEL_NAME + "_WTI" + str(sparsity_value), _)
  brent_train, brent_test = read_model_results(MODEL_NAME + "_BRENT" + str(sparsity_value), _)

  print("sparsity: ", sparsity_value, "trial: ", _)
  print("WTI")
  show_models_results_with_labels(wti_test_outputs, wti_test)
  print("Brent")
  show_models_results_with_labels(brent_test_outputs, brent_test)
  print("-"*20)
  print

sparsity:  25 trial:  0
WTI
MAE:  0.9877703656898654
RMSE:  2.485675957951343
MSE:  6.178584967937328
MAPE:  0.02362054297929181
R2:  0.9589976402191077
Brent
MAE:  0.9116918871165645
RMSE:  1.3608099439699817
MSE:  1.851803703607585
MAPE:  0.01747625382126151
R2:  0.9889944526361812
--------------------
sparsity:  25 trial:  1
WTI
MAE:  1.3443888901098902
RMSE:  2.887000458799998
MSE:  8.3347716491114
MAPE:  0.033622730849452886
R2:  0.9446887422246562
Brent
MAE:  0.965591472392638
RMSE:  1.4362455461492905
MSE:  2.0628012688336734
MAPE:  0.018969663963417562
R2:  0.987740462435588
--------------------
sparsity:  25 trial:  2
WTI
MAE:  1.0301488156288154
RMSE:  2.6300182503836402
MSE:  6.916995997351025
MAPE:  0.026708472848996603
R2:  0.9540973928564326
Brent
MAE:  0.9262677865030676
RMSE:  1.3828113717198764
MSE:  1.9121672897578061
MAPE:  0.01798165958020245
R2:  0.9886357027831962
--------------------
sparsity:  25 trial:  3
WTI
MAE:  1.0443691159951156
RMSE:  2.5488033740754994
M